In [15]:
from openbb_optimization.models.optimizer_query_params import OptimizerQueryParams
from openbb_optimization.aggregator import _get_multi_asset_data
from openbb_optimization.features.mean_variance import _mpt_optimize
from pypfopt import expected_returns, risk_models
from pypfopt.efficient_frontier import EfficientFrontier
import numpy as np

In [2]:
params = OptimizerQueryParams(
    symbols=["AMZN","NVDA","F","AAL","PEP","JNJ"],
    start_date="2021-12-30",
    end_date="2024-12-30",
    risk_free_rate=0.0,
    max_weight=1.0
)

In [3]:
df = _get_multi_asset_data(params)
df

/home/cjpellicci/workspace/openbb_extension_dev/.testvenv/lib/python3.10/site-packages/openbb_core/app/command_runner.py:139: OpenBBWarning: Parameter 'chart' not found.
  warn(
/home/cjpellicci/workspace/openbb_extension_dev/.testvenv/lib/python3.10/site-packages/openbb_core/app/command_runner.py:139: OpenBBWarning: Parameter 'chart' not found.
  warn(
/home/cjpellicci/workspace/openbb_extension_dev/.testvenv/lib/python3.10/site-packages/openbb_core/app/command_runner.py:139: OpenBBWarning: Parameter 'chart' not found.
  warn(
/home/cjpellicci/workspace/openbb_extension_dev/.testvenv/lib/python3.10/site-packages/openbb_core/app/command_runner.py:139: OpenBBWarning: Parameter 'chart' not found.
  warn(
/home/cjpellicci/workspace/openbb_extension_dev/.testvenv/lib/python3.10/site-packages/openbb_core/app/command_runner.py:139: OpenBBWarning: Parameter 'chart' not found.
  warn(
/home/cjpellicci/workspace/openbb_extension_dev/.testvenv/lib/python3.10/site-packages/openbb_core/app/command

,AMZN,NVDA,F,AAL,PEP,JNJ
date,,,,,,
2021-12-30,168.6445,29.586,20.47,18.07,172.67,172.31
2021-12-31,166.7170,29.411,20.77,17.96,173.71,171.07
2022-01-03,170.4045,30.121,21.77,18.75,172.98,171.54
2022-01-04,167.5220,29.290,24.31,19.02,173.23,171.08
2022-01-05,164.3570,27.604,23.66,18.68,173.82,172.22
...,...,...,...,...,...,...
2024-12-23,225.0600,139.670,9.90,17.25,151.29,145.27
2024-12-24,229.0500,140.220,10.02,17.35,152.81,145.85
2024-12-26,227.0500,139.930,10.07,17.35,152.44,145.58


In [4]:
returns = df.pct_change().dropna()
returns

,AMZN,NVDA,F,AAL,PEP,JNJ
date,,,,,,
2021-12-31,-0.011429,-0.005915,0.014656,-0.006087,0.006023,-0.007196
2022-01-03,0.022118,0.024141,0.048146,0.043987,-0.004202,0.002747
2022-01-04,-0.016916,-0.027589,0.116674,0.014400,0.001445,-0.002682
2022-01-05,-0.018893,-0.057562,-0.026738,-0.017876,0.003406,0.006664
2022-01-06,-0.006711,0.020794,0.033812,-0.005889,0.000230,-0.003426
...,...,...,...,...,...,...
2024-12-23,0.000622,0.036897,0.002024,0.021919,-0.009817,0.005537
2024-12-24,0.017729,0.003938,0.012121,0.005797,0.010047,0.003993
2024-12-26,-0.008732,-0.002068,0.004990,0.000000,-0.002421,-0.001851


In [6]:
returns.dtypes


AMZN    float64
NVDA    float64
F       float64
AAL     float64
PEP     float64
JNJ     float64
dtype: object

In [7]:
s = risk_models.sample_cov(returns, frequency=252)
s

,AMZN,NVDA,F,AAL,PEP,JNJ
AMZN,23673.207773,-415.207894,-190.980623,-744.536869,402.082869,-713.651422
NVDA,-415.207894,585884.525776,-201.377111,61.901123,684.053933,-1032.769044
F,-190.980623,-201.377111,11825.307256,222.655457,-370.637163,-1292.618547
AAL,-744.536869,61.901123,222.655457,11924.008776,160.617867,81.138918
PEP,402.082869,684.053933,-370.637163,160.617867,90295.721822,2489.055855
JNJ,-713.651422,-1032.769044,-1292.618547,81.138918,2489.055855,34246.020022


In [19]:
print(returns.isin([np.inf, -np.inf]).values.any(), returns.isna().values.any())

False False


In [21]:
mu = expected_returns.mean_historical_return(returns, frequency=252, returns_data=True)
mu

AMZN    0.095199
NVDA    0.672172
F      -0.216343
AAL    -0.008404
PEP    -0.042363
JNJ    -0.059744
dtype: float64

In [25]:
ef = EfficientFrontier(mu, s)
ef